Restarted Python 3.11.4

In [1]:
from pathlib import Path
import re
import dbf
import csv
import json
from datetime import date

# Primary path
PATH = Path('C:/dbase/pyScripts/eBay/')

# eBay report csv
input_file = Path('/'.join([str(PATH),'18Report.csv']))
# dBase AD Table
dBase_file = Path('/'.join([str(PATH),'ad.dbf']))
ad = dbf.Table(filename=str(dBase_file))
ad.open(dbf.READ_WRITE)

# dBase fieldnames that need to be populated
# and modules that we sell. Populated fields will
# be returned as function values
with open(Path('/'.join([str(PATH), 'fields.json']))) as fields_file:
    fields = json.load(fields_file)
modules = [
    'S-100+',
    'SP-40W+',
    'STK0050+',
    'STK0059+',
    'STK0080+',
    'STK1060+',
    'STK1070+',
    'TA-100W+',
    'TA-100WA+',
    'TA-200W+'
]

# Price breakdown (Parts/Labor/Shipping) 
# of all modules that we sell
with open(Path('/'.join([str(PATH), 'Module_Price_List.json']))) as price_file:
    price_list = json.load(price_file)
    

# Ingest csv reports from eBay and 
# return dBase record formatted data
def order_parser(row):

    def name_sort(order):
        # Split buyer name into "first" and "last" names
        return {
            'first': order['Buyer Name'].split(' ')[0].upper(),
            'last': order['Buyer Name'].split(' ')[0-1].upper()
        }
    
    def address(order):
        # Define address according to dBase record format
        return {
            'street': ' '.join([order['Ship To Address 1'], order['Ship To Address 2']]).upper(),
            'city' : order['Ship To City'].upper(),
            'state': order['Ship To State'].upper(),
            'zip': order['Ship To Zip'].split('-')[0]
        }

    def item_name(order):
        # If the item is one of our TA/STK Modules,
        # return the module name
        # Return tech/warr fields as appropriate
        if order['Item Title'].split(' ')[0] in modules:
            return {
                'make': 'MINE',
                'model': order['Item Title'].split(' ')[0],
                'tech': 'C',
                'warranty': ''
            }
        # If the item is an ODR Outer, the naming
        # format is a bit different
        elif order['Item Title'].split(' ')[0:2] == ['Pioneer', 'ODR']:
            return {
                'make': 'MINE',
                'model': 'ODR Outer',
                'tech': 'C',
                'warranty': ''
            }
        # If we sold something else entirely,
        # "MINE" will not be the "MAKE" value
        else:
            return {
                'make': order['Item Title'].split(' ')[0].upper(),
                'model': order['Item Title'].split(' ')[1],
                'tech': '',
                'warranty': 'PARTS'
            }

    def complaint(order):
        # Simple string that shows qty of items
        # for dBase records
        if int(order['Quantity']) == 1:
            module = 'MODULE'
        elif int(order['Quantity']) > 1:
            module = 'MODULES'
        return '  '.join([' ', order['Quantity'], item_name(order)['model'], module])
    
    def module_pricing(item_name):
        # Price breakdown (Parts/Labor/Shipping) 
        # of all modules that we sell
        return price_list[item_name]

    def field_trim(field, item):
        field_length = len(getattr(ad[0], field))
        if len(item) > field_length:
            return item[:field_length]
        else:
            return item

    # Return a copy of a record's fields with all
    # relevant fields populated
    fields['FIRST'] = field_trim('FIRST', name_sort(row)['first'])
    fields['LAST'] = field_trim('LAST', name_sort(row)['last'])
    fields['STREET'] = field_trim('STREET', address(row)['street'])
    fields['CITY'] = field_trim('CITY', address(row)['city'])
    fields['STATE'] = field_trim('STATE', address(row)['state'])
    fields['ZIP'] = address(row)['zip']
    fields['MAKE'] = field_trim('MAKE', item_name(row)['make'])
    model = item_name(row)['model']
    fields['MODEL'] = field_trim('MODEL', model)
    fields['COMP'] = complaint(row)
    fields['TECH'] = item_name(row)['tech']
    today = dbf.Date(date.today())
    fields['DATE_IN'] = today
    fields['DATE_OUT'] = today
    fields['WARR'] = item_name(row)['warranty']
    fields['PAID_LABOR'] = module_pricing(model)['labor'] * float(row['Quantity'])
    fields['PAID_PARTS'] = module_pricing(model)['parts'] * float(row['Quantity'])
    fields['NOTAX'] = module_pricing(model)['ship'] * float(row['Quantity'])
    fields['AMT_DEP'] = module_pricing(model)['total'] * float(row['Quantity'])
    return fields

In [2]:
# open eBay report
order_format = re.compile(r'^\d{2}-\d{5}-\d{5}$')    
with open(input_file,'r') as csvread:
    next(csvread)
    orders = csv.DictReader(csvread)
    for line in orders:
        if line['Order Number'] is not None \
        and order_format.match(line['Order Number']):
            print(line)

{'Sales Record Number': '979', 'Order Number': '18-11228-92736', 'Buyer Username': 'alexowright', 'Buyer Name': 'alex O wright', 'Buyer Email': '35d1803d373d92183889@members.ebay.com', 'Buyer Note': '', 'Buyer Address 1': '8615 Jackson Springs Rd', 'Buyer Address 2': '', 'Buyer City': 'Tampa', 'Buyer State': 'FL', 'Buyer Zip': '33615-3207', 'Buyer Country': 'United States', 'Buyer Tax Identifier Name': '', 'Buyer Tax Identifier Value': '', 'Ship To Name': 'alex wright', 'Ship To Phone': '+1 813-360-4557', 'Ship To Address 1': '8615 Jackson Springs Rd', 'Ship To Address 2': '', 'Ship To City': 'Tampa', 'Ship To State': 'FL', 'Ship To Zip': '33615-3207', 'Ship To Country': 'United States', 'Item Number': '126054497095', 'Item Title': 'STK0050+   Pioneer  SX-780 or marantz  SR4000  replaces  STK0050 STK0030 STK0040', 'Custom Label': '', 'Sold Via Promoted Listings': 'No', 'Quantity': '2', 'Sold For': '$64.99', 'Shipping And Handling': '$0.00', 'Item Location': 'Austin, Texas', 'Item Zip C

In [3]:
len(ad)

3

In [4]:
ad.append(line)

FieldMissingError: 'SALES RECORD NUMBER:  no such field in table'

In [5]:
print(ad[0])

  0 - REFERRAL  : 'MFG       '
  1 - TICKET    : 141487
  2 - SHIPPER   : '                         '
  3 - FIRST     : 'THOMAS                '
  4 - LAST      : 'LAHR                  '
  5 - STREET    : '715 BREEZY LAKE WAY             '
  6 - CITY      : 'ELDERSBURG     '
  7 - STATE     : 'MD '
  8 - ZIP       : 21784
  9 - PHONE1    : 410.7900189
 10 - PHONE2    : 410.5499511
 11 - EMAIL     : 'thomas.lahr@comcast.net  '
 12 - MAKE      : 'HEGEL          '
 13 - MODEL     : 'H590           '
 14 - SERIAL    : '10A407              '
 15 - DEALER    : '                    '
 16 - DLR_CITY  : '          '
 17 - DOP       : None
 18 - COMP      : '  NO AUDIO'
 19 - EXP       : ''
 20 - PARTS     : ''
 21 - COMMENTS  : '                                                  '
 22 - DATE_IN   : datetime.date(2024, 3, 28)
 23 - DATE_OUT  : None
 24 - TECH      : '   '
 25 - WARR      : '          '
 26 - PAID_LABOR: None
 27 - WLABOR    : None
 28 - PAID_PARTS: None
 29 - NOTAX     : None
 3

In [6]:
print(ad[0]['FIRST'])

THOMAS                
